# Rag Techniques

This notebook walks through the process of building RAG app(s) from scratch. They will build towards a broader understanding of the RAG landscape, as shown below. Note this notebook is a refactor of the [OpenAI Rag From Scratch repo](https://github.com/langchain-ai/rag-from-scratch) but simplified, and modified to support Bedrock.

![RAG Landscape - full](images/rag_landscape_full.png)

## Environment Setup

`(1) Packages`

In [22]:
! pip install -qU langchain_community tiktoken langchain_aws langchainhub chromadb langchain arxiv pymupdf langgraph 
! pip install -qU matplotlib scikit-learn pandas umap
! pip install -qU ragatouille



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.

Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

no such option: -U


`(2) LangSmith`

Setting the following LangSmith environment variables allows the use of [LangSmith tracing](https://smith.langchain.com/). To use this you need a LangSmith API key (requires a free account creating). This is optional.

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

`(3) AWS Credentials`

In [3]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = '<UPDATE_THIS>'
os.environ["AWS_SECRET_ACCESS_KEY"] = '<UPDATE_THIS>'
os.environ["AWS_SESSION_TOKEN"] = '<UPDATE_THIS>'
os.environ["AWS_REGION"] = 'us-west-2'

`(4) Imports`

In [18]:
from bs4 import BeautifulSoup as Soup
from IPython.display import Image, display
from langchain import hub
from langchain_aws import BedrockEmbeddings, ChatBedrock
from langchain_community.document_loaders import WebBaseLoader, ArxivLoader, RecursiveUrlLoader
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain_community.utilities import SQLDatabase
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.globals import set_debug
from langchain.load import dumps, loads
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.utils.math import cosine_similarity
from langgraph.graph import START, StateGraph
from operator import itemgetter
from pprint import pprint
from pydantic import BaseModel, Field
from ragatouille import RAGPretrainedModel
from ragatouille.utils import get_wikipedia_page
from sklearn.mixture import GaussianMixture
from typing import Literal, Optional, Dict, List, Optional, Tuple
from typing_extensions import Annotated, TypedDict
import bs4
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import umap
import uuid



## Basic RAG
### Basic Indexing

![indexing](images/indexing.png)

Let's define a simple question and document to seed the database with:

In [5]:
#### INDEXING - documents ####

question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

[Tiktoken](https://github.com/openai/tiktoken/blob/main/README.md) is a fast [BPE](https://en.wikipedia.org/wiki/Byte_pair_encoding) open-source tokenizer by OpenAI. Given a text string (e.g., `tiktoken is great!`) and an encoding (e.g., `cl100k_base`), a tokenizer can split the text string into a list of tokens (e.g., [`t`, `ik`, `token`, `is`, `great`, `!`]). Splitting text strings into tokens is useful because GPT models see text in the form of tokens. Knowing how many tokens are in a text string can tell you (a) whether the string is too long for a text model to process and (b) how much an OpenAI API call costs (as usage is priced by token).

In [ ]:
#### INDEXING - utility function ####

import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

Data needs embedding first before it can be used by an LLM and/or vector data store. The [BedrockEmbeddings](https://python.langchain.com/api_reference/aws/embeddings/langchain_aws.embeddings.bedrock.BedrockEmbeddings.html#langchain_aws.embeddings.bedrock.BedrockEmbeddings) uses `amazon.titan-embed-text-v1` by default.

In [ ]:
#### INDEXING - embeddings ####

embeddings = BedrockEmbeddings()
query_result = embeddings.embed_query(question)
document_result = embeddings.embed_query(document)
len(query_result)

[DocumentLoaders](https://python.langchain.com/docs/integrations/document_loaders/) load data into the standard LangChain Document format. Each DocumentLoader has its own specific parameters, but they can all be invoked in the same way with the .load method.



In [8]:
#### INDEXING - loading documents ####

# Load blog

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

Before loading the documents into vector store they need to be [split](https://python.langchain.com/docs/how_to/recursive_text_splitter/). Here we are starting off with basic splitting by length (`300` characters with `50` overlap), but later will explore other splitting techniques.

In [9]:
#### INDEXING - splitting ####

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

Then finally we can load the split embeddings into the [vector store](https://python.langchain.com/docs/integrations/vectorstores/). For simplicity we are using [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/) as the vector store.

In [10]:
#### INDEXING - splitting ####

vector_store = Chroma.from_documents(documents=splits, 
                                    embedding=BedrockEmbeddings())

retriever = vector_store.as_retriever(search_kwargs={"k": 1})

### Basic Retrieval

Using a `retriever` we can query the vector store.

In [ ]:
#### RETRIEVAL ####

docs = retriever.invoke("What is Task Decomposition?")

print("No. of results: ", len(docs))
docs

### Basic Generation

![generation](images/generation.png)

Define the prompt we will pass to the LLM.

In [ ]:
#### GENERATION - prompt ####

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt_basic = ChatPromptTemplate.from_template(template)
prompt_basic

Define the LLM.

> Note that langchain does not yet seem to support the new Amazon `Nova` models (see [JIRA](https://github.com/langchain-ai/langchain-aws/issues/308) ticket). Once support has been added we can try Nova models.

In [229]:
#### GENERATION - llm ####

llm_claude_3_5_sonnet_v2 = ChatBedrock(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=os.environ["AWS_SESSION_TOKEN"], 
    region_name=os.environ["AWS_REGION"],
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    model_kwargs={"temperature": 0}
)

Build  a basic chain. The `|` operator [chains runnable objects](https://python.langchain.com/docs/how_to/sequence/) (objects that have an `invoke()` function) together so as one object is streaming output, the next object in the chain can receive the stream as input.

In [14]:
#### GENERATION - basic chain ####

chain_basic = prompt_basic | llm_claude_3_5_sonnet_v2

Execute the chain by calling its invoke method. The `dict` passed to `invoke()` is used to tokenize varibles declared at any of the steps.

In [ ]:
#### GENERATION - basic chain ####

chain_basic.invoke({"context":docs,"question":"What is Task Decomposition?"})

Instead of defining our own prompts, we can make use of prompt templates published in the [Langchain Hub](https://smith.langchain.com/hub). Lets replace our previous prompt with one from the hub and rebuild the chain.

In [ ]:
#### GENERATION - basic chain using langchain hub prompt ####

prompt_basic = hub.pull("rlm/rag-prompt")
pprint(prompt_basic)

chain_basic = prompt_basic | llm_claude_3_5_sonnet_v2
chain_basic.invoke({"context":docs, "question":"What is Task Decomposition?"})


We can now build a basic RAG chain, where instead of explicitly passing `docs` as the context we instead provide the `retriever` to query the vector store directly.

In [ ]:
#### GENERATION - basic rag chain ####

chain_basic_rag = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_basic
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

chain_basic_rag.invoke("What is Task Decomposition?")

## Query Translation
### Multi Query

![multi-query](images/multi-query.png)

Basic RAG chains find similar embedded documents based on a distance metric. But, retrieval may produce different results with subtle changes in query wording, or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The [MultiQueryRetriever](https://python.langchain.com/docs/how_to/MultiQueryRetriever/) automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever can mitigate some of the limitations of the distance-based retrieval and get a richer set of results.

First let's build a prompt that enables multi-query.

In [18]:
#### QUERY TRANSLATION - multi-query prompt ####

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

chain_multi_query_generate_queries = (
    prompt_perspectives 
    | llm_claude_3_5_sonnet_v2   
    | StrOutputParser() 
    | (lambda x: x.split("\n\n"))
)

Once we have the results from the 5 different variations of the original query, we need to union the results.

In [19]:
#### QUERY TRANSLATION - multi-query utility function ####

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

We can now build a second chain that joins the results.

In [ ]:
#### QUERY TRANSLATION - multi-query retrieve ####

chain_multi_query_retrieval = chain_multi_query_generate_queries | retriever.map() | get_unique_union

question = "What is task decomposition for LLM agents?"
docs = chain_multi_query_retrieval.invoke({"question":question})

print("No. of results: ", len(docs))
docs


Take a look at the [Langchain tracing](https://smith.langchain.com/) for the request we just executed. You will see the call to the LLM was provided the following input:

```
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: What is task decomposition for LLM agents?
```

And rendered the following output:

```
Here are 5 alternative versions of the question to help retrieve relevant documents:

How do LLM agents break down complex tasks into smaller subtasks?

What are the methods and techniques used for decomposing tasks when working with language model agents?

Can you explain the process of splitting larger problems into manageable steps for AI agents?

What is the role of task planning and decomposition in LLM-based autonomous systems?

How do large language model agents analyze and structure tasks into hierarchical components?
```

It then made 5 queries to the vector store to answer those questions, and joined (union) the results.

Finally, we build a third chain once has retrieved the results from the previous chain, uses those documents as context to answer the original question:


In [ ]:
#### QUERY TRANSLATION - multi-query RAG ####

chain_multi_query_rag = (
    {"context": chain_multi_query_retrieval, 
     "question": itemgetter("question")} 
    | prompt_basic
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

chain_multi_query_rag.invoke({"question":question})


Taking a look at the [Langchain tracing](https://smith.langchain.com/) again for the request we just executed, you will see two calls. The first call is the same as the one we just previously described. The second call provides the results from the vector store as context to the LLM to answer the original question.


### RAG-Fusion

![rag fusion](images/rag_fusion.png)

RAG-Fusion takes multi-query one step further. It employs multiple query generation just like multi-query, but then uses Reciprocal Rank Fusion to re-ran the search results.

Let's start by defining the prompt and chain to retrieve related documents:

In [22]:
#### QUERY TRANSLATION - RAG-Fusion: Related ####

template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

chain_rag_fusion_generate_queries = (
    prompt_rag_fusion 
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser() 
    | (lambda x: x.split("\n\n"))
)


As mentioned previously, RAG-Fusion applies a reciprocal rank fusion function to rerank the results.

In [23]:
#### QUERY TRANSLATION - utility function ####

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

chain_rag_fusion_retrieval = chain_rag_fusion_generate_queries | retriever.map() | reciprocal_rank_fusion


In [ ]:
docs = chain_rag_fusion_retrieval.invoke({"question": question})
print(len(docs))
docs

We can now build the final chain that once has retrieved the results for the different variations of the query, then applies the re-ranking function.

In [25]:
#### QUERY TRANSLATION - RAG-Fusion: RAG chain ####

chain_rag_fusion_retrieval_rag = (
    {"context": chain_rag_fusion_retrieval, 
     "question": itemgetter("question")} 
    | prompt_basic
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)


In [ ]:
chain_rag_fusion_retrieval_rag.invoke({"question":question})

Inspect the [Langchain tracing](https://smith.langchain.com/) to understand the flow better.

### Decomposition

_Decomposition_ breaks down a question into multiple sub-questions, then proceeds to answer each question before having the context to answer the original question.

There are 2 variations - answering the decomposed questions recursively, or answering individually. Let's start by looking at the recursive option first.

![decomposition_recursive](images/decomposition_recursive.png)



Arxiv papers:

- [Least-to-Most Prompting Enables Complex Reasoning in Large Language Models](https://arxiv.org/pdf/2205.10625)
- [Interleaving Retrieval with Chain-of-Thought Reasoning for Knowledge-Intensive Multi-Step Questions](https://arxiv.org/pdf/2212.10509)

Let's start by creating the prompt and chain to generate the initial decomposed sub-questions:


In [27]:
#### QUERY TRANSLATION - Decomposition - decompose question ####

template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""

prompt_decomposition_decompose = ChatPromptTemplate.from_template(template)

chain_decomposition_generate_queries = ( 
                                        prompt_decomposition_decompose 
                                        | llm_claude_3_5_sonnet_v2
                                        | StrOutputParser() 
                                        | (lambda x: x.split("\n\n")))

In [ ]:
#### QUERY TRANSLATION - Decomposition - decompose question ####

question = "What are the main components of an LLM-powered autonomous agent system?"
decomposed_questions = chain_decomposition_generate_queries.invoke({"question":question})

decomposed_questions

Define the prompt to answer the original question based on the additional context gathered via the decomposition chain:

In [29]:
#### QUERY TRANSLATION - Decomposition - recursive prompt ####

template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

prompt_decomposition_solve_recursively = ChatPromptTemplate.from_template(template)

Define an additional chain that takes the `decomposed_questions` and solves using the above prompt.

In [30]:
#### QUERY TRANSLATION - Decomposition - utility function ####

def format_qa_pair_recursive(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

In [31]:
#### QUERY TRANSLATION - Decomposition - RAG recursive ####

def decomposition_retrieve_and_rag_recursive(decomposed_questions):
    q_a_pairs = ""
    for q in decomposed_questions:
        
        rag_chain = (
            {"context": itemgetter("question") | retriever, 
            "question": itemgetter("question"),
            "q_a_pairs": itemgetter("q_a_pairs")} 
            | prompt_decomposition_solve_recursively
            | llm_claude_3_5_sonnet_v2
            | StrOutputParser())

        answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
        q_a_pair = format_qa_pair_recursive(q,answer)
        q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair
        
    return (answer, q_a_pairs)

In [ ]:
answer,q_a_pairs = decomposition_retrieve_and_rag_recursive(decomposed_questions)

print(answer)
print(q_a_pairs)

Let's now take a look at how to perform decomposition individually.

![decomposition_individually](images/decomposition_individually.png)

In [33]:
#### QUERY TRANSLATION - Decomposition - RAG individually ####

def decomposition_retrieve_and_rag_individually(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.invoke(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (
            prompt_rag 
            | llm_claude_3_5_sonnet_v2 
            | StrOutputParser()).invoke({"context": retrieved_docs, "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

In [ ]:
#### QUERY TRANSLATION - Decomposition - RAG individually ####

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = decomposition_retrieve_and_rag_individually(question, prompt_basic, chain_decomposition_generate_queries)

print("Answers:")
print(answers)

print("Questions:")
print(questions)

Build the final chain to perform decompostion individually.

In [38]:
#### QUERY TRANSLATION - Decomposition - utility function ####

def format_qa_pairs_individually(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs_individually(questions, answers)

In [39]:
#### QUERY TRANSLATION - Decomposition - prompt an chain individually ####

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt_decomposition_solve_individually = ChatPromptTemplate.from_template(template)

chain_decomposition_solve_individually = (
    prompt_decomposition_solve_individually
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

In [ ]:
result=chain_decomposition_solve_individually.invoke({"context":context,"question":question})

pprint(result)

### Step Back

Step Back prompting enables LLMs to step back and look at the bigger picture, finding general ideas and riles from specific examples. Using these ideas, the LLM can solve problems more logically and make fewer mistakes.

![step back](images/step%20back.png)

Arxiv papers:

- [Take a Step Back: Evoking Reasoning via Abstraction in Large Language Models](https://arxiv.org/pdf/2310.06117)

Let's start by defining the prompt which includes few shot examples:

In [41]:
#### QUERY TRANSLATION - Step back - prompts ####

# Few Shot Examples
step_back_examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
prompt_step_back_example = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
prompt_step_back_few_shot = FewShotChatMessagePromptTemplate(
    example_prompt=prompt_step_back_example,
    examples=step_back_examples,
)
prompt_step_back_abstract = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        prompt_step_back_few_shot,
        # New question
        ("user", "{question}"),
    ]
)

In [42]:
chain_step_back_generate_queries = prompt_step_back_abstract | llm_claude_3_5_sonnet_v2 | StrOutputParser()


In [ ]:
question = "What is task decomposition for LLM agents?"
chain_step_back_generate_queries.invoke({"question": question})

In [44]:
#### QUERY TRANSLATION - Step back - RAG ####

# Response prompt 
template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
prompt_step_back_solve = ChatPromptTemplate.from_template(template)

chain_step_back_rag = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": chain_step_back_generate_queries | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | prompt_step_back_solve
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)



In [ ]:
chain_step_back_rag.invoke({"question": question})

### HyDE

HyDE (Hypothetical Document Embeddings) is an embedding technique that takes queries, generates a hypothetical answer, and then embeds that generated document and uses that as the final example.

![hyde](images/hyde.png)

Arxiv paper:

- [Precise Zero-Shot Dense Retrieval without Relevance Labels](https://arxiv.org/pdf/2212.10496)

Let's start by building the prompts and chain to retrieve the documents:


In [46]:
#### QUERY TRANSLATION - HyDE - retrieval prompt and chain ####

template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""

prompt_hyde = ChatPromptTemplate.from_template(template)

chain_hyde_generate_docs_for_retrieval = (
    prompt_hyde | llm_claude_3_5_sonnet_v2 | StrOutputParser() 
)

In [ ]:
question = "What is task decomposition for LLM agents?"
chain_hyde_generate_docs_for_retrieval.invoke({"question":question})

Next build the retriever chain:

In [ ]:
#### QUERY TRANSLATION - HyDE - retriever chain ####

chain_hyde_retriever = chain_hyde_generate_docs_for_retrieval | retriever 

hyde_retrieved_docs = chain_hyde_retriever.invoke({"question":question})
hyde_retrieved_docs

And finally build the RAG chain:

In [ ]:
#### QUERY TRANSLATION - HyDE - RAG ####

chain_hyde_rag = (
    prompt_basic
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

chain_hyde_rag.invoke({"context":hyde_retrieved_docs,"question":question})

## Routing
### Logical routing

Logical routing uses function-calling for classification.

![logical routing](images/logical_routing.png)

In [85]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
# Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema, 
# so you don't need to worry about the model omitting a required key, or hallucinating an invalid enum value.
structured_llm_route_query = llm_claude_3_5_sonnet_v2.with_structured_output(RouteQuery)

# Prompt 
template = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt_routing_logical = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", "{question}"),
    ]
)

# Define router 
chain_routing_logical = prompt_routing_logical | structured_llm_route_query

In [ ]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result_routing_logical = chain_routing_logical.invoke({"question": question})

result_routing_logical

We can then use a [custom function to route between different sub chains](https://python.langchain.com/docs/how_to/routing/#using-a-custom-function-recommended) based on the response.

In [53]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"


chain_routing_logical_full = chain_routing_logical | RunnableLambda(choose_route)

In [ ]:
chain_routing_logical_full.invoke({"question": question})

### Semantic routing

Semantic routing involves using embeddings to route a query to the most relevant prompt based on semantic similarity.

In [55]:
# Two prompts
template_routing_semantic_physics = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

template_routing_semantic_math = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
templates_routing_semantic = [template_routing_semantic_physics, template_routing_semantic_math]
prompt_embeddings = embeddings.embed_documents(templates_routing_semantic)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = templates_routing_semantic[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == template_routing_semantic_math else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain_routing_semantic = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

In [ ]:
chain_routing_semantic.invoke("What's a black hole")

## Query Construction

With typical RAG, a user query is converted into a vector representation. This vector is then compared to vector representations of the source documents to find the most similar ones. This works fairly well for unstructured data. Query construction is the process of converting natural language into a specific query syntax for each structured data type.

Examples include [Text-to-metadata-filter](https://python.langchain.com/docs/how_to/self_query/) for Vectorstores, [Text-to-SQL](https://python.langchain.com/docs/tutorials/sql_qa/) for SQL DB, [Text-to-SQL+ Semantic](https://github.com/langchain-ai/langchain/blob/master/cookbook/retrieval_in_sql.ipynb?ref=blog.langchain.dev) for PGVector supported SQL DB, and [Text-to-Cypher](https://neo4j.com/labs/neodash/2.4/user-guide/extensions/natural-language-queries/) for graph databases.

### Text-to-metadata-filter

Vectorstores equipped with metadata filtering enable structured queries to filter embedded unstructured documents. The self-query retriever can translate natural language queries into these structured queries using a few steps:

![text-to-metadata-filter](images/text-to-metadata-filter.png)

Let's use Arxiv papers as an example:


In [ ]:
docs_text_to_metadata_filter = ArxivLoader(
    query="reasoning",
    load_max_docs=5,
    load_all_available_meta=True
).load()

print(docs_text_to_metadata_filter[0].page_content[:1000])
print(docs_text_to_metadata_filter[0].metadata)

Let's assume we want to build an index that enables us to:

- perform unstructured search over the `Title` and `Summary` attributes of each document
- use range filtering on `Published`

To convert a natural langauge query into a structured query we need to define a schema for the structured search queries:

In [79]:
class ArxivSearch(BaseModel):
    """Search over Arxiv documents."""

    title_search: str = Field(
        ...,
        description="Similarity search query applied to the title.",
    )
    summary_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to summaries. "
            "Should be succinct and only include key words that could be in a summary."
        ),
    )
    earliest_published_date: Optional[datetime.date] = Field(
        None,
        description="Earliest published date filter, inclusive. Only use if explicitly specified.",
    )
    latest_published_date: Optional[datetime.date] = Field(
        None,
        description="Latest published date filter, exclusive. Only use if explicitly specified.",
    )


    def pretty_print(self) -> None:
        for field in self.model_fields:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.model_fields[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

Now we prompt the LLM to produce queries.

In [86]:
text_to_metadata_filter_system_template = """You are an expert at converting user questions into database queries. \
You have access to a database of scholarly articles. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
text_to_metadata_filter_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", text_to_metadata_filter_system_template),
        ("human", "{question}"),
    ]
)

text_to_metadata_filter_structured_llm = llm_claude_3_5_sonnet_v2.with_structured_output(ArxivSearch)

chain_text_to_metadata_filter_query_analyzer = text_to_metadata_filter_prompt | text_to_metadata_filter_structured_llm

In [ ]:
chain_text_to_metadata_filter_query_analyzer.invoke({"question": "rag from scratch"}).pretty_print()

In [ ]:
chain_text_to_metadata_filter_query_analyzer.invoke({"question": "what papers on RAG were published in 2024?"}).pretty_print()

### Text-to-sql 

Text-to-SQL is a task in natural language processing (NLP) where the goal is to automatically generate SQL queries from natural language text. The task involves converting the text input into a structured representation and then using this representation to generate a semantically correct SQL query that can be executed on a database.

Let's start by seeding a local SQLLite database with sample data:


In [89]:
!curl -s https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql | sqlite3 Chinook.db

In [ ]:
text_to_sql_db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(text_to_sql_db.dialect)
print(text_to_sql_db.get_usable_table_names())
text_to_sql_db.run("SELECT * FROM Artist LIMIT 10;")



Create a class to map state (keep track of input question, generated query, query result, and generated answer):

In [94]:
class TextToSqlState(TypedDict):
    question: str
    query: str
    result: str
    answer: str

The first step is to take the user input and convert it to a SQL query.

In [ ]:
template = """Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific 
number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}
"""

prompt_text_to_sql = ChatPromptTemplate.from_template(template)
prompt_text_to_sql

# Note: the langchain-ai/sql-query-system-prompt" prompt is not compatible with Bedrock as it passes the prompt as `system` instead of `prompt`
# prompt_text_to_sql = hub.pull("langchain-ai/sql-query-system-prompt")


The prompt includes several parameters we will need to populate, such as the SQL dialect and table schemas. LangChain's [SQLDatabase](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html) object includes methods to help with this. Our `text_to_sql_write_query` step will just populate these parameters and prompt a model to generate the SQL query:

In [ ]:
set_debug(False)

class TextToSqlQueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]
    
def text_to_sql_write_query(state: TextToSqlState):
    """Generate SQL query to fetch information."""
    prompt = prompt_text_to_sql.invoke(
        {
            "dialect": text_to_sql_db.dialect,
            "top_k": 10,
            "table_info": text_to_sql_db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm_claude_3_5_sonnet_v2.with_structured_output(TextToSqlQueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

text_to_sql_write_query({"question": "How many Employees are there?"})


This is the most dangerous part of creating a SQL chain. Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution.

To execute the query, we will load a tool from [langchain-community](https://python.langchain.com/docs/concepts/architecture/#langchain-community). Our `text_to_sql_execute_query` node will just wrap this tool:

In [202]:
def text_to_sql_execute_query(state: TextToSqlState):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=text_to_sql_db)
    return {"result": execute_query_tool.invoke(state["query"])}

Our last step generates an answer to the question given the information pulled from the database:

In [102]:
def text_to_sql_generate_answer(state: TextToSqlState):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm_claude_3_5_sonnet_v2.invoke(prompt)
    return {"answer": response.content}

We can now build the final chain:

In [ ]:
text_to_sql_graph_builder = StateGraph(TextToSqlState).add_sequence(
    [text_to_sql_write_query, text_to_sql_execute_query, text_to_sql_generate_answer]
)
text_to_sql_graph_builder.add_edge(START, "text_to_sql_write_query")
text_to_sql_graph = text_to_sql_graph_builder.compile()

display(Image(text_to_sql_graph.get_graph().draw_mermaid_png()))


Let's test our application! Note that we can stream the results of individual steps:


In [ ]:
for step in text_to_sql_graph.stream(
    {"question": "How many employees are there?"}, stream_mode="updates"
):
    print(step)


### Text-to-cypher 

Text-to-cypher is a task in natural language processing (NLP) where the goal is to automatically generate graph queries from natural language text. Similar to text-to-sql the task involves converting the text input into a structured representation and then using this representation to generate a semantically correct SQL query that can be executed on a database.

> **TODO!**: A text-to-cypher example will required Neo4j installing. Figure out simplest way to do this.

## Indexing

### Multi-representation Indexing

![Multi-representation Indexing](images/Multi-representation%20Indexing.png)

Multi-representation Indexing in LLMs organizes data using multiple types of representations, such as text, images, or structured formats, to improve how information is retrieved and understood. By encoding data into different embedding spaces optimized for specific tasks or modalities, this approach enables powerful capabilities like semantic search, cross-modal retrieval (e.g., finding images using text queries), and task-specific optimizations. For instance, a technician could describe a problem in text and instantly retrieve related guides, images, or videos. This technique leverages the strengths of LLMs and other models to handle diverse data types, making retrieval more accurate, flexible, and scalable for real-world applications.

Arxiv paper:

- [Dense X Retrieval: What Retrieval Granularity Should We Use?](https://arxiv.org/pdf/2312.06648)

For this example we will use a `MultiVectorRetriever` that retrieves raw documents from an `InMemoryByteStore`, and their summaries from a vector store (Chroma).

Let's start by downloading and summarizing docs to seed the database with:

In [219]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs_multi_representation_indexing = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs_multi_representation_indexing.extend(loader.load())

chain_multi_representation_indexing = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

summaries_multi_representation_indexing = chain_multi_representation_indexing.batch(docs_multi_representation_indexing, {"max_concurrency": 5})

Next, store the summaries in the vector store and the raw documents in the byte store.

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore_multi_representation_indexing = Chroma(collection_name="summaries",
                     embedding_function=embeddings)

# The storage layer for the parent documents
byte_store_multi_representation_indexing = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever_multi_representation_indexing = MultiVectorRetriever(
    vectorstore=vectorstore_multi_representation_indexing,
    byte_store=byte_store_multi_representation_indexing,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs_multi_representation_indexing = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries_multi_representation_indexing)
]

# Add
retriever_multi_representation_indexing.vectorstore.add_documents(summary_docs_multi_representation_indexing)
retriever_multi_representation_indexing.docstore.mset(list(zip(doc_ids, docs_multi_representation_indexing)))

Let's try querying both the summary and raw docs:

In [ ]:
query = "Memory in agents"
sub_docs = vectorstore_multi_representation_indexing.similarity_search(query,k=1)
sub_docs[0]

In [ ]:
retrieved_docs = retriever_multi_representation_indexing.invoke(query,n_results=1)
retrieved_docs[0].page_content[0:500]

When splitting documents for retrieval, there are often conflicting desires:

- You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
- You want to have long enough documents that the context of each chunk is retained.

An alternative to the approach we just implemented for multi-representation indexing is the [ParentDocumentRetriever](https://python.langchain.com/docs/how_to/parent_document_retriever/) that strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

Note that "parent document" refers to the document that a small chunk originated from. This can either be the whole raw document OR a larger chunk.

### RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval

RAPTOR enhances language models by organizing information into a hierarchical tree structure. It starts by dividing documents into small text chunks, which are then embedded, clustered based on similarity, and summarized. This process repeats recursively, creating multiple layers of summaries that capture information at varying levels of detail. When responding to queries, RAPTOR retrieves relevant information from different levels of this tree, allowing the model to integrate both detailed and high-level context. This method has demonstrated significant improvements in tasks requiring complex reasoning, achieving state-of-the-art results in benchmarks like QuALITY.

![raptor](images/raptor.png)

Links:

- [RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval](https://arxiv.org/pdf/2401.18059) paper
- [Deep dive video](https://www.youtube.com/watch?v=jbGchdTL7d0)
- [Code repo](https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb)

Using the RAPTOR code repo as inspiration, lets obtain the LCEL docs to seed the database:

In [ ]:
def raptor_load_docs():
    # LCEL docs
    url = "https://python.langchain.com/docs/expression_language/"
    loader = RecursiveUrlLoader(
        url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs = loader.load()

    # LCEL w/ PydanticOutputParser (outside the primary LCEL docs)
    url = "https://python.langchain.com/docs/how_to/output_parser_structured/"
    loader = RecursiveUrlLoader(
        url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs_pydantic = loader.load()

    # LCEL w/ Self Query (outside the primary LCEL docs)
    url = "https://python.langchain.com/docs/how_to/self_query/"
    loader = RecursiveUrlLoader(
        url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs_sq = loader.load()

    # Doc texts
    docs.extend([*docs_pydantic, *docs_sq])
    docs_texts = [d.page_content for d in docs]
    
    # Doc texts concat
    docs_sorted = sorted(docs, key=lambda x: x.metadata["source"])
    docs_reversed = list(reversed(docs_sorted))
    concatenated_content = "\n\n\n --- \n\n\n".join(
        [doc.page_content for doc in docs_reversed]
    )
    print(
        "Num tokens in all context: %s"
        % num_tokens_from_string(concatenated_content, "cl100k_base")
    )
    
    chunk_size_tok = 2000
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size_tok, chunk_overlap=0
    )
    texts_split = text_splitter.split_text(concatenated_content)
    return (docs_texts, texts_split)
    
raptor_load_docs()

#### Tree Construction

The clustering approach in tree construction includes a few interesting ideas.

**GMM (Gaussian Mixture Model)**

- Model the distribution of data points across different clusters
- Optimal number of clusters by evaluating the model's Bayesian Information Criterion (BIC)

**UMAP (Uniform Manifold Approximation and Projection)**

- Supports clustering
- Reduces the dimensionality of high-dimensional data
- UMAP helps to highlight the natural grouping of data points based on their similarities

**Local and Global Clustering**

- Used to analyze data at different scales
- Both fine-grained and broader patterns within the data are captured effectively

**Thresholding**

- Apply in the context of GMM to determine cluster membership
- Based on the probability distribution (assignment of data points to ≥ 1 cluster)

Let's define the functions ([original source](https://github.com/parthsarthi03/raptor/blob/master/raptor/cluster_tree_builder.py)) we will need to build the tree:

In [ ]:
RANDOM_SEED = 224  # Fixed seed for reproducibility

def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters

Define additional functions ([original source](https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb)):

In [266]:
def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embeddings.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # Summarization
    template = """Here is a sub-set of LangChain Expression Language doc. 
    
    LangChain Expression Language provides a way to compose chain in LangChain.
    
    Give a detailed summary of the documentation provided.
    
    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm_claude_3_5_sonnet_v2 | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results

We can now build the tree:

In [ ]:
(leaf_texts, texts_split) = raptor_load_docs()

results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

The paper reports best performance from collapsed tree retrieval.

This involves flattening the tree structure into a single layer and then applying a k-nearest neighbors (kNN) search across all nodes simultaneously.

We do simply do this below.

In [269]:
# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)

# Now, use all_texts to build the vectorstore with Chroma
vectorstore_raptor = Chroma.from_texts(texts=all_texts, embedding=embeddings)
retriever_raptor = vectorstore_raptor.as_retriever()

Now we can using our flattened, indexed tree in a RAG chain.

In [ ]:
# Prompt
prompt_raptor = hub.pull("rlm/rag-prompt")

# Post-processing
def format_docs_raptor(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
chain_raptor = (
    {"context": retriever | format_docs_raptor, "question": RunnablePassthrough()}
    | prompt_raptor
    | llm_claude_3_5_sonnet_v2
    | StrOutputParser()
)

# Question
chain_raptor.invoke("How to define a RAG chain? Give me a specific code example.")

### ColBERT: Contextualized Late Interaction over BERT

> **Note:** Attempting to load the pretrained ColBERT model is failing on my Mac M1 with `RuntimeError: Error building extension 'segmented_maxsim_cpp'`. Have not been able to find a fix for this yet, therefore the code in this section may fail.

ColBERT is a neural retrieval model that enhances search efficiency and effectiveness by combining deep language model representations with scalable retrieval techniques. Unlike traditional models that encode queries and documents into single vectors, ColBERT generates multiple vectors at the token level, allowing for fine-grained interactions between queries and documents. This late interaction mechanism enables ColBERT to pre-compute document representations offline, significantly speeding up query processing. By leveraging vector-similarity search indexes, ColBERT can perform end-to-end retrieval directly from large document collections, achieving high recall and precision while maintaining computational efficiency. 

To explain more clearly: You embed the query and the passage and get vector representation for every token in both. Then, for each query token, you find the token in the passage with the largest dot product (i.e. the largest similarity). This is called the “maxsim” for each token. Finally, the similarity score between the query and the passage is the summation of all the maxsims you just found

The [RAGatouille](https://python.langchain.com/docs/integrations/retrievers/ragatouille/) library includes support for ColBERT.


Links:

- [ColBERTv2: Effective and Efficient Retrieval via Lightweight Late Interaction)](https://arxiv.org/pdf/2112.01488) paper

To create an index, you'll need to load a trained model, this can be one of your own or a pretrained one from the hub! Creating an index with the default configuration is just a few lines of code:

In [19]:
rag_colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
docs_colbert = [get_wikipedia_page("Hayao_Miyazaki"), get_wikipedia_page("Studio_Ghibli")]
colbert_index_path = rag_colbert.index(
    collection=docs_colbert,
    index_name="my_index", 
    max_document_length=180,
    split_documents=True,
)

[Dec 28, 16:20:04] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


Using /Users/deanhart/Library/Caches/torch_extensions/py311_cpu as PyTorch extensions root...
No modifications detected for re-loaded extension module segmented_maxsim_cpp, skipping build step...
Loading extension module segmented_maxsim_cpp...


ImportError: dlopen(/Users/deanhart/Library/Caches/torch_extensions/py311_cpu/segmented_maxsim_cpp/segmented_maxsim_cpp.so, 0x0002): tried: '/Users/deanhart/Library/Caches/torch_extensions/py311_cpu/segmented_maxsim_cpp/segmented_maxsim_cpp.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/deanhart/Library/Caches/torch_extensions/py311_cpu/segmented_maxsim_cpp/segmented_maxsim_cpp.so' (no such file), '/Users/deanhart/Library/Caches/torch_extensions/py311_cpu/segmented_maxsim_cpp/segmented_maxsim_cpp.so' (no such file)

You can also optionally add document IDs or document metadata when creating the index:

In [20]:
doc_ids = ["miyazaki", "ghibli"]
doc_metadatas = [
    {"entity": "person", "source": "wikipedia"},
    {"entity": "organisation", "source": "wikipedia"},
]
colbert_index_path = rag_colbert.index(
    index_name="my_index_with_ids_and_metadata",
    collection=docs_colbert,
    document_ids=doc_ids,
    document_metadatas=doc_metadatas,
)


NameError: name 'rag_colbert' is not defined

Once this is done running, your index will be saved on-disk and ready to be queried! RAGatouille and ColBERT handle everything here:

- Splitting your documents
- Tokenizing your documents
- Identifying the individual terms
- Embedding the documents and generating the bags-of-embeddings
- Compressing the vectors and storing them on disk

Once an index is created, querying it is just as simple as creating it! You can either load the model you need directly from an index's configuration:


In [ ]:
results = rag_colbert.search(query="What animation studio did Miyazaki found?", k=3)

RAG.search is a flexible method! You can set the k value to however many results you want (it defaults to 10), and you can also use it to search for multiple queries at once:

In [ ]:
rag_colbert.search(["What manga did Hayao Miyazaki write?",
                    "Who are the founders of Ghibli?"
                    "Who is the director of Spirited Away?"],)

We can also search using a retriever:

In [ ]:
retriever_colbert = rag_colbert.as_langchain_retriever(k=3)
retriever_colbert.invoke("What animation studio did Miyazaki found?")

## Retrieval

### Re-ranking

![re-ranking](images/re-ranking.png)

We touched on this subject earlier when we looked at RAG-Fusion where we implemented a recipricol rank fusion function to re-rank results. But lets look at how to achieve the same using the recently released _rerank_ model from Amazon.